In [ ]:
import pandas as pd
from IPython.display import clear_output
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')
servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=servico, options=options)
lista_nomes, lista_promocionais, lista_originais, lista_descontos = [], [], [], []

def retira_ponto(valor):
    """
    Remove pontos de um valor fornecido como string.
    Args:
        valor (str): Valor a ser tratado.
    Returns:
        str: Valor sem pontos.
    """
    valor = valor.replace('.', '')
    return valor


def calcular_preco(precos_inteiros, precos_centavos, tem_desconto):
    """
    Calcula o preço original, o preço com desconto e a porcentagem de desconto de um produto.

    A função processa os valores fornecidos em listas contendo os preços inteiros e centavos de um produto,
    além de um indicador que informa se há desconto. Com base nesses valores, ela calcula o preço original,
    o preço com desconto (se aplicável) e a porcentagem de desconto.

    Args:
        precos_inteiros (list): Lista contendo os elementos representando a parte inteira dos preços.
                                Deve ser uma lista de objetos retornados pelo Selenium.
        precos_centavos (list): Lista contendo os elementos representando os centavos dos preços.
                                Deve ser uma lista de objetos retornados pelo Selenium.
        tem_desconto (bool): Indica se o produto possui desconto. Se `True`, o preço com desconto é calculado.

    Returns:
        tuple: Uma tupla contendo:
               - preco_original (float): O preço original do produto.
               - preco_desconto (float): O preço com desconto do produto (ou o preço original se não houver desconto).
               - desconto (float): A porcentagem de desconto aplicada ao produto.
    """
    try:
        preco_original = float(f'{retira_ponto(precos_inteiros[0].text)}.{precos_centavos[0].text}')
    except IndexError:
        preco_original = float(f'{retira_ponto(precos_inteiros[0].text)}')

    if tem_desconto:
        try:
            preco_desconto = float(f'{retira_ponto(precos_inteiros[1].text)}.{precos_centavos[1].text}')
        except IndexError:
            preco_desconto = float(f'{retira_ponto(precos_inteiros[1].text)}')
    else:
        preco_desconto = preco_original

    desconto = f'{(100 - ((preco_desconto / preco_original) * 100)):.2f}'

    return preco_original, preco_desconto, float(desconto)


def pegar_itens(link, dicionario):
    """
    Coleta informações de produtos a partir de uma página da web fornecida.

    A função utiliza Selenium para realizar scraping dos dados de produtos em uma página específica.
    Ela captura o nome do produto, preço original, preço com desconto e a porcentagem de desconto
    dos produtos exibidos na página. Os dados coletados são adicionados ao dicionário fornecido.

    Args:
        link (str): URL da página da web que contém os produtos a serem coletados.
        dic (dict): Dicionário onde os dados coletados dos produtos serão armazenados.
                    Deve conter as chaves:
                    - 'Nome': Lista de nomes dos produtos.
                    - 'Preco Original': Lista de preços originais.
                    - 'Preco Com Desconto': Lista de preços promocionais.
                    - 'Porcentagem de Desconto': Lista de porcentagens de desconto.

    Returns:
        None: Os dados são armazenados diretamente no dicionário fornecido como argumento.
    """
    driver.get(link)

    cards = driver.find_elements(By.CLASS_NAME, 'poly-card')

    for i, card in enumerate(cards):
        clear_output(wait=True)
        print(f'registrando as informações do produto {i+1}')

        nome = card.find_element(By.CLASS_NAME, 'poly-component__title').text
        precos_inteiros = card.find_elements(By.CLASS_NAME, 'andes-money-amount__fraction')[:2]
        precos_centavos = card.find_elements(By.CLASS_NAME, 'andes-money-amount__cents')[:2]

        tem_desconto = True if card.find_elements(By.CLASS_NAME, 'andes-money-amount--previous') else False
        preco_original, preco_desconto, desconto = calcular_preco(precos_inteiros, precos_centavos, tem_desconto)

        lista_originais.append(preco_original)
        lista_promocionais.append(preco_desconto)
        lista_descontos.append(desconto)
        lista_nomes.append(nome)

    dicionario['Nome'] = lista_nomes
    dicionario['Preco Original'] = lista_originais
    dicionario['Preco Com Desconto'] = lista_promocionais
    dicionario['Porcentagem de Desconto'] = lista_descontos


def converter_resultados():
    """
    Extrai informações de produtos de múltiplas páginas de ofertas e as converte em um DataFrame.

    A função utiliza Selenium para acessar várias páginas de ofertas de produtos no site Mercado Livre.
    Ela percorre todas as páginas disponíveis, coleta informações detalhadas dos produtos utilizando
    a função `pegar_itens` e organiza esses dados em um DataFrame para fácil manipulação e análise.

    Args:
        None

    Returns:
        pandas.DataFrame: Um DataFrame contendo os dados dos produtos coletados, com as colunas:
                          - 'Nome': Nome do produto.
                          - 'Preco Original': Preço original do produto.
                          - 'Preco Com Desconto': Preço promocional do produto.
                          - 'Porcentagem de Desconto': Porcentagem de desconto aplicada ao produto.
    """
    driver.get('https://www.mercadolivre.com.br/ofertas?page=1')

    paginas = driver.find_elements(By.CLASS_NAME, 'andes-pagination__link')
    pag_final = int(paginas[-2].text)

    produtos = {}

    for pag in range(1, pag_final + 1):
        clear_output(wait=True)
        print(f'Pegando os produtos da página {pag}')
        link = f'https://www.mercadolivre.com.br/ofertas?page={pag}'
        pegar_itens(link, produtos)

    clear_output(wait=True)
    driver.quit()
    print(f'{len(produtos['Nome'])} produtos encontrados em {pag} páginas.')

    return pd.DataFrame.from_records(produtos)

In [148]:
produtos_df = converter_resultados()
top_descontos = produtos_df.sort_values(['Porcentagem de Desconto'], ascending=False)[:5]
display(top_descontos)

1033 produtos encontrados em 20 páginas.


,Nome,Porcentagem de Desconto,Preco Com Desconto,Preco Original
155,Kit Tupia Manual Laminadora 800w 6mm + Jogo De...,67.00,221.69,671.80
868,Speaker Extreme Britânia Bbs200bt Flash Lights...,61.40,215.83,559.10
880,Relógio Inteligente Smart Watch Blackview Lant...,60.00,279.99,699.98
433,Colchão Emma Original Casal - Tecnologia Alemã...,59.05,2199.00,5369.90
862,Cafeteira Nespresso Vertuo Pop Cor Branco 110v,59.04,319.23,779.39
